In [6]:
# 返回指令集出现次数
import re
import collections
import pandas
def read_s(file_name):
    """
    读取指令集文件，并返回指令集出现次数的字典
    :param file_name: 指令集文件名
    :return: 指令集出现次数的字典
    """
    instruction_pattern = re.compile(r'\b(\w+)\b')
    dict_s = {}
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match1 = instruction_pattern.search(line)
            if  match1:
                instruction = match1.group(1)
                if dict_s.get(instruction) is None:
                    dict_s[instruction] = 1
                else:
                    dict_s[instruction] += 1
    return dict_s

In [7]:
# 此函数用来处理o文件，统计各数字字母出现次数
import collections
def read_o(filename):
    with open(filename, 'rb') as file:
        exe_content = file.read()
    # 输出前100个字节的十六进制表示
    text = exe_content.hex().upper()
    dict_o = dict(collections.Counter(text))
    return dict_o

In [8]:
# 此函数用于切割s文件
import os
import pandas as pd
def split_by_colon(input_file,add):
    if not os.path.exists(f"{add}"):
        os.mkdir(f"{add}")
    if not os.path.exists(f"{add}/"+input_file):
        os.mkdir(f"{add}/"+input_file.split(".s")[0])
    path = os.path.join(f"{add}/",input_file.split(".s")[0])
    with open('s/'+input_file, 'r') as f:
        lines = f.readlines()

    # 初始化一个空字典，用于存储冒号后的内容
    parts = {}
    key = "pre"
    # 遍历每一行
    for line in lines:
        # 找到冒号的位置
        colon_index = line.find(":")
        if colon_index != -1 and ".ident" not in line and ".ascii" not in line:
            # 如果找到了冒号，则将冒号后的内容保存到字典中
            key = line[:colon_index].strip()  # 冒号前的部分作为键
        if parts.get(key) is None:
            parts[key] = []
        parts[key].append(line)

    # 逐个保存冒号后的内容到单独的文件中
    for key, value in parts.items():
        output_file = os.path.join(path,key+".txt")  # 根据冒号前的内容构造文件名
        with open(output_file, 'w') as f:
            for txt in value:
                f.write(txt)

In [9]:
# 统计s文件各字符出现次数,保存为csv文件
import pandas as pd
import os
def one_data(name:str,floder_name):
    # 输入的字典列表
    dict_list = []
    name_list = os.listdir(floder_name)
    for name_l in name_list:
        if name in name_l:
            temp = read_s(f"{floder_name}/{name_l}")
            temp1 = read_o(f"{floder_name}/{name_l}".replace('s','o'))
            temp.update(temp1)
            temp["version"] = name_l.split("_")[1][:2]
            # print(temp)
            dict_list.append(temp)
    df = pd.DataFrame.from_dict(dict_list)
    df = df.fillna(0)
    df.to_csv(f'{name}.csv', index=False)  # 如果不希望保存索引列，可以将 index 参数设置为 False

In [10]:
# 用于提取特征，保存到附件1.csv
one_data('附件2_','s')

# 这段先不用运行
file_name = os.listdir("s")
for name in file_name:
    if "附件1" in name:
        split_by_colon(name,"附件1")
    if "附件2" in name:
        split_by_colon(name,"附件2")